In [2]:
#Exploring logistic regression

include("readclassjson.jl")
data = readclassjson("titanic.json")
v_train = data["Vtrain"]
v_test = data["Vtest"]
U_train = data["Utrain"]
U_test = data["Utest"]

178×7 Matrix{Float64}:
 3.0  1.0  26.0  0.0  0.0    7.925   2.0
 3.0  0.0  20.0  0.0  0.0    8.05    2.0
 3.0  1.0  14.0  0.0  0.0    7.8542  2.0
 2.0  1.0  55.0  0.0  0.0   16.0     2.0
 3.0  1.0  15.0  0.0  0.0    8.0292  1.0
 3.0  1.0  28.0  0.0  0.0    7.8792  1.0
 1.0  1.0  28.0  1.0  0.0  146.521   0.0
 3.0  1.0  28.0  0.0  0.0    7.75    1.0
 3.0  1.0  28.0  0.0  0.0    7.75    1.0
 3.0  0.0  28.0  2.0  0.0   21.6792  0.0
 1.0  0.0  65.0  0.0  1.0   61.9792  0.0
 3.0  0.0  28.5  0.0  0.0    7.2292  0.0
 3.0  0.0  11.0  5.0  2.0   46.9     2.0
 ⋮                           ⋮       
 3.0  0.0   2.0  4.0  1.0   39.6875  2.0
 3.0  1.0  15.0  1.0  0.0   14.4542  0.0
 3.0  0.0  28.0  0.0  0.0    7.2292  0.0
 3.0  0.0  21.0  0.0  0.0    8.6625  2.0
 3.0  0.0  34.5  0.0  0.0    6.4375  0.0
 3.0  0.0  28.0  8.0  2.0   69.55    2.0
 3.0  0.0  74.0  0.0  0.0    7.775   2.0
 3.0  1.0   9.0  1.0  1.0   15.2458  0.0
 2.0  1.0  42.0  0.0  0.0   13.0     2.0
 3.0  0.0  26.0  0.0  0.0    7.8958  

In [3]:
function confusionMatrix(y_hat, y)
    cm = zeros(2, 2)
    for i in 1:size(y, 1)
        row = Int(y_hat[i])
        col = Int(y[i])
        cm[row, col] += 1
    end
    return cm
end

n_test = size(v_test, 1)
n_train = size(v_train, 1)

v_test_hat_a = zeros(n_test)

for i in 1:n_test
    if U_test[i, 2] == 0
        v_test_hat_a[i] = 1
    else
        v_test_hat_a[i] = 2
    end
end    

In [4]:
cm_a = confusionMatrix(v_test_hat_a, v_test)

2×2 Matrix{Float64}:
 83.0  23.0
 15.0  57.0

In [5]:
v_test_hat_b = zeros(n_test)

for i in 1:n_test
    if U_test[i, 1] < 3
        v_test_hat_b[i] = 2
    else
        v_test_hat_b[i] = 1
    end
end    

In [7]:
cm_b = confusionMatrix(v_test_hat_b, v_test)

2×2 Matrix{Float64}:
 72.0  26.0
 26.0  54.0

In [8]:
err_a = (cm_a[1, 2] + cm_a[2, 1])/n_test
err_b = (cm_b[1, 2] + cm_b[2, 1])/n_test

println(err_a)
println(err_b)

0.21348314606741572
0.29213483146067415


In [9]:
using Flux: onehotbatch
using LinearAlgebra

embarked_train = transpose(onehotbatch(U_train[:, 7], 0:2))
embarked_test = transpose(onehotbatch(U_test[:, 7], 0:2))

alone_train = zeros(n_train)
alone_test = zeros(n_test)

for i in 1:n_train
    if U_train[i, 4] == 0 && U_train[i, 5] == 0
        alone_train[i] = 1
    end
end

for i in 1:n_test
    if U_test[i, 4] == 0 && U_test[i, 5] == 0
        alone_test[i] = 1
    end
end

X_train = hcat(U_train[:, 1:6], embarked_train, alone_train)
X_test = hcat(U_test[:, 1:6], embarked_test, alone_test)

178×10 Matrix{Float64}:
 3.0  1.0  26.0  0.0  0.0    7.925   0.0  0.0  1.0  1.0
 3.0  0.0  20.0  0.0  0.0    8.05    0.0  0.0  1.0  1.0
 3.0  1.0  14.0  0.0  0.0    7.8542  0.0  0.0  1.0  1.0
 2.0  1.0  55.0  0.0  0.0   16.0     0.0  0.0  1.0  1.0
 3.0  1.0  15.0  0.0  0.0    8.0292  0.0  1.0  0.0  1.0
 3.0  1.0  28.0  0.0  0.0    7.8792  0.0  1.0  0.0  1.0
 1.0  1.0  28.0  1.0  0.0  146.521   1.0  0.0  0.0  0.0
 3.0  1.0  28.0  0.0  0.0    7.75    0.0  1.0  0.0  1.0
 3.0  1.0  28.0  0.0  0.0    7.75    0.0  1.0  0.0  1.0
 3.0  0.0  28.0  2.0  0.0   21.6792  1.0  0.0  0.0  0.0
 1.0  0.0  65.0  0.0  1.0   61.9792  1.0  0.0  0.0  0.0
 3.0  0.0  28.5  0.0  0.0    7.2292  1.0  0.0  0.0  1.0
 3.0  0.0  11.0  5.0  2.0   46.9     0.0  0.0  1.0  0.0
 ⋮                           ⋮                      
 3.0  0.0   2.0  4.0  1.0   39.6875  0.0  0.0  1.0  0.0
 3.0  1.0  15.0  1.0  0.0   14.4542  1.0  0.0  0.0  0.0
 3.0  0.0  28.0  0.0  0.0    7.2292  1.0  0.0  0.0  1.0
 3.0  0.0  21.0  0.0  0.0  

In [13]:
using Statistics

X_train_mean = mean(X_train, dims=1)
X_train_std = std(X_train, dims=1)

X_train_s = (X_train .- X_train_mean) ./ X_train_std
X_test_s = (X_test .- X_train_mean) ./ X_train_std

X_train_final = hcat(ones(n_train), X_train_s)
X_test_final = hcat(ones(n_test), X_test_s)

178×11 Matrix{Float64}:
 1.0   0.825661   1.39991   -0.258767   …  -0.292192   0.596966   0.814966
 1.0   0.825661  -0.713328  -0.730015      -0.292192   0.596966   0.814966
 1.0   0.825661   1.39991   -1.20126       -0.292192   0.596966   0.814966
 1.0  -0.362689   1.39991    2.01893       -0.292192   0.596966   0.814966
 1.0   0.825661   1.39991   -1.12272        3.4176    -1.67278    0.814966
 1.0   0.825661   1.39991   -0.101684   …   3.4176    -1.67278    0.814966
 1.0  -1.55104    1.39991   -0.101684      -0.292192  -1.67278   -1.22532
 1.0   0.825661   1.39991   -0.101684       3.4176    -1.67278    0.814966
 1.0   0.825661   1.39991   -0.101684       3.4176    -1.67278    0.814966
 1.0   0.825661  -0.713328  -0.101684      -0.292192  -1.67278   -1.22532
 1.0  -1.55104   -0.713328   2.80435    …  -0.292192  -1.67278   -1.22532
 1.0   0.825661  -0.713328  -0.0624133     -0.292192  -1.67278    0.814966
 1.0   0.825661  -0.713328  -1.43689       -0.292192   0.596966  -1.22532
 ⋮   

In [14]:
using Flux
using Flux: logitcrossentropy, params

function multi_logistic(X, Y, reps)
    # data sizes
    n,d = size(X)
    m = size(Y,2)
    
    # linear predictor parameter
    theta = zeros(d,m)

    # predictor
    predicty(x) = theta'*x
    margin(pi, pj, y) = (2*dot(pi-pj,y) + dot(pj,pj) - dot(pi,pi)) / (2*norm(pi-pj) + 1e-10)
    multilogisticloss(yhat, y) = sum([exp(margin(r, y, yhat)) for r in reps])
    
    lambda = 0.01
    rlas(theta) = sum(abs.(theta[2:end]))
    loss(x,y) = multilogisticloss(predicty(x), y) + lambda*rlas(theta)

    data = zip(eachrow(X), eachrow(Y))
    opt = ADAMW()
    Flux.@epochs 500 Flux.train!(loss, params(theta), data, opt)
    return theta
end

function classification_regression_fit(X, Y, l, r, lambda, reps; numiters = 500)
    #     Inputs:
    #         X: input data
    #         Y: output data
    #         l: l(yhat, y)
    #         r: r(theta)
    #         lambda: regularization hyper-parameter
    #         numiters (optional): number of iterations
        
        data = zip(eachrow(X), eachrow(Y))
        n,d = size(X)
        theta = zeros(d)
        predicty(x) = theta'*x
        loss(x, y) = l(predicty(x), y) + lambda*r(theta)
        cost(x,y) = loss(x,y) + lambda*r(theta)
        risk() = sum((cost(d...) for d in data))/n
        opt = Flux.ADAGrad()
        losses = []
        tracker() = push!(losses, risk())
        Flux.@epochs numiters Flux.train!(loss, Flux.params(theta), data, opt, cb = Flux.throttle(tracker,10))
        return theta
    end

lambda = 0.01
rlas(theta) = sum(abs.(theta[2:end]))
margin(pi, pj, y) = (2*dot(pi-pj,y) + dot(pj,pj) - dot(pi,pi)) / (2*norm(pi-pj) + 1e-10)
logisticloss(yhat, y) = sum([exp(margin(r, y, yhat)) for r in reps])

logisticloss (generic function with 1 method)

In [15]:
k1 = [1]
k2 = [2]
reps = [k1, k2]

theta = multi_logistic(X_train_final, v_train, reps)

┌ Info: Epoch 1
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 2
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 3
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 4
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 5
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 6
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 7
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 8
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 9
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 10
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 11
└ @ Main C:\Users\surface\.julia

┌ Info: Epoch 104
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 105
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 106
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 107
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 108
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 109
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 110
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 111
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 112
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 113
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 114
└ @ Main C:\

┌ Info: Epoch 206
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 207
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 208
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 209
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 210
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 211
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 212
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 213
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 214
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 215
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 216
└ @ Main C:\

┌ Info: Epoch 308
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 309
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 310
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 311
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 312
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 313
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 314
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 315
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 316
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 317
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 318
└ @ Main C:\

┌ Info: Epoch 410
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 411
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 412
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 413
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 414
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 415
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 416
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 417
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 418
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 419
└ @ Main C:\Users\surface\.julia\packages\Flux\6Q5r4\src\optimise\train.jl:154
┌ Info: Epoch 420
└ @ Main C:\

11×1 Matrix{Float64}:
  1.1508154666627337
 -0.3759754659464131
  0.6119142531844808
 -0.2141115426515621
 -0.22834419253825872
 -0.09583047286438935
  0.10450924018372665
  0.059168179251369996
 -0.016201723509365495
 -0.010467488479245772
 -0.15682077897979246

In [16]:
theta

11×1 Matrix{Float64}:
  1.1508154666627337
 -0.3759754659464131
  0.6119142531844808
 -0.2141115426515621
 -0.22834419253825872
 -0.09583047286438935
  0.10450924018372665
  0.059168179251369996
 -0.016201723509365495
 -0.010467488479245772
 -0.15682077897979246

In [18]:
psiinv(yhat) = reps[argmin([norm(yhat-reps[k][1]) for k=1:length(reps)])][1]

function unembed(v_em)
    n = size(v_em, 1)
    v_hat = zeros(n)
    for i in 1:n
        v_hat[i] = psiinv(v_em[i])
    end
    return v_hat
end

unembed (generic function with 1 method)

In [19]:
v_train_em = X_train_final * theta
v_train_hat = unembed(v_train_em)
cm_train = confusionMatrix(v_train_hat, v_train)

v_test_em = X_test_final * theta
v_test_hat = unembed(v_test_em)
cm_test = confusionMatrix(v_test_hat, v_test)

display(cm_train)
display(cm_test)

2×2 Matrix{Float64}:
 392.0   81.0
  59.0  179.0

2×2 Matrix{Float64}:
 84.0  22.0
 14.0  58.0